In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import sys
sys.path.insert(0,'/content/drive/MyDrive/ColabNotebooks/')

In [ ]:
%pip install lion-pytorch

In [ ]:
%pip install torchmetrics

In [ ]:
import torch
from torch import optim
from torchmetrics.classification import BinaryF1Score, BinaryAccuracy, BinarySpecificity, BinaryPrecision, BinaryRecall
from model.wide_res_net import WideResNet
from model.smooth_cross_entropy import smooth_crossentropy
from coronary import Coronary
from utility.log import Log
from utility.initialize import initialize
from lion_pytorch import Lion
from utility.step_lr import StepLR


In [ ]:
def train(swa_start, model, swa_model, epoch, device, train_loader, optimizer, log, scheduler, swa_scheduler):
    model.train()
    log.train(len_dataset=len(dataset.train))

    for batch in train_loader:
        inputs, targets = (b.to(device) for b in batch)
        inputs = inputs.permute(0, 3, 2, 1)

        optimizer.zero_grad()

        predictions = model(inputs)
        loss = smooth_crossentropy(predictions.logits, targets, smoothing=0.0001)
        loss.mean().backward()

        optimizer.step()

        with torch.no_grad():
            correct = torch.argmax(predictions.logits, 1) == targets
            log(model, loss.cpu(), correct.cpu(), scheduler.get_last_lr()[0])

In [ ]:
def test(model, device, test_loader, log):
    model.eval()
    log.eval(len_dataset=len(test_loader))

    with torch.no_grad():
        for batch in test_loader:
            inputs, targets = (b.to(device) for b in batch)
            inputs = inputs.permute(0, 3, 2, 1)

            predictions = model(inputs)
            loss = smooth_crossentropy(predictions, targets, smoothing=0.0001)
            correct = torch.argmax(predictions, 1) == targets
            log(model, loss.cpu(), correct.cpu())

In [ ]:
if __name__ == "__main__":

    num_epochs = 50
    # model_name_path = '/content/drive/MyDrive/ColabNotebooks/results_logs/'
    model_name = 'inceptionv3-swa'
    # model_name = model_name_path + model_name
    initialize(seed=111)
    log = Log(log_each=1, log_name=model_name)

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    model = torch.hub.load('pytorch/vision:v0.10.0', 'inception_v3', pretrained=True).to(device)
    model.num_classes = 2
    model.eval()

    dataset = Coronary(batch_size=100, threads=2, img_size_definition=299)
    optimizer = torch.optim.SGD(model.parameters(), lr=0.0004, momentum=0.0, weight_decay=0.4)

    swa_model = torch.optim.swa_utils.AveragedModel(model)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=50)
    swa_start = 25
    swa_scheduler = torch.optim.swa_utils.SWALR(optimizer, swa_lr=0.001)

    for epoch in range(num_epochs):
        train(swa_start, model, swa_model, epoch, device, dataset.train, optimizer, log, scheduler, swa_scheduler)
        test(model, device, dataset.valid, log)
        if epoch > swa_start:
          swa_model.update_parameters(model)
          swa_scheduler.step()
        else:
          scheduler.step()

    log.flush()
    # torch.save(model.state_dict(), '../model_trained/'+model_name+'.pth')
    #68% 54% -valid

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0


100
300
880
┏━━━━━━━━━━━━━━┳━━━━━━━╸T╺╸R╺╸A╺╸I╺╸N╺━━━━━━━┳━━━━━━━╸S╺╸T╺╸A╺╸T╺╸S╺━━━━━━━┳━━━━━━━╸V╺╸A╺╸L╺╸I╺╸D╺━━━━━━━┓
┃              ┃              ╷              ┃              ╷              ┃              ╷              ┃
┃       epoch  ┃        loss  │    accuracy  ┃        l.r.  │     elapsed  ┃        loss  │    accuracy  ┃
┠──────────────╂──────────────┼──────────────╂──────────────┼──────────────╂──────────────┼──────────────┨
┃           0  ┃      7.1450  │      0.01 %  ┃   4.000e-04  │       00:11  ┃      7.1180  │      0.00 %  ┃
┃           1  ┃      6.6360  │      0.01 %  ┃   3.996e-04  │       00:11  ┃      6.9779  │      0.00 %  ┃
┃           2  ┃      6.1639  │      0.02 %  ┃   3.984e-04  │       00:11  ┃      6.3201  │      0.00 %  ┃
┃           3  ┃      5.7033  │      0.07 %  ┃   3.965e-04  │       00:11  ┃      5.7725  │      0.33 %  ┃
┃           4  ┃      5.2950  │      0.16 %  ┃   3.937e-04  │       00:11  ┃      5.5270  │      2.67 %  ┃
┃           5  ┃      4.8

In [ ]:
metricF1 = BinaryF1Score().to(device)
metricAcc = BinaryAccuracy().to(device)
metricSpecificity = BinarySpecificity().to(device)
metricPrecision = BinaryPrecision().to(device)
metricRecall = BinaryRecall().to(device)

In [ ]:
inputs = dataset.test_x.permute(0, 3, 2, 1).to(device)
predictions = model(inputs)
f1 = metricF1(torch.argmax(predictions, 1), dataset.test_y.to(device))
acc = metricAcc(torch.argmax(predictions, 1), dataset.test_y.to(device))
specificity = metricSpecificity(torch.argmax(predictions, 1), dataset.test_y.to(device))
precision = metricPrecision(torch.argmax(predictions, 1), dataset.test_y.to(device))
recall = metricRecall(torch.argmax(predictions, 1), dataset.test_y.to(device))

with open(model_name+'.txt', 'a') as file:
                file.write(f"\n \n F1: {f1}\n Accuracy: {acc}\n Specificity:{specificity}\n Precision: {precision}\n Recall: {recall}")

print(f1)
print(acc)
print(precision)
print(specificity)
print(recall)
